# 2.5 Model registry

In [5]:
from mlflow import MlflowClient

## Mlflow client

In [6]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

### Access experiments

In [7]:
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1732646263960, experiment_id='1', last_update_time=1732646263960, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1732645905274, experiment_id='0', last_update_time=1732645905274, lifecycle_stage='active', name='Default', tags={}>]

### Create new experiment

In [8]:
# client.create_experiment(name='test-experiment-creation')

### Access runs

In [26]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',                     # id experiment (e.g. nyc-taxi regressor)
    filter_string='',                       # filter query
    run_view_type=ViewType.ACTIVE_ONLY,     # to access active only or active and deleted
    max_results=5,                          # max number of runs to visualize
    order_by=['metrics.rmse ASC'],          # possible to include multiple criteria
)

In [10]:
# first run
runs[0]

<Run: data=<RunData: metrics={'rmse': 5.218622162238321}, params={'learning_rate': '0.05716128214396281',
 'max_depth': '83',
 'min_child_weight': '2.8929928124399993',
 'objective': 'reg:squarederror',
 'reg_alpha': '0.06533442718772015',
 'reg_lambda': '0.0028303214779712895',
 'seed': '42'}, tags={'mlflow.runName': 'abrasive-slug-522',
 'mlflow.source.name': '/home/codespace/.local/lib/python3.12/site-packages/ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'codespace',
 'model': 'xgboost'}>, info=<RunInfo: artifact_uri='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/9b8e79d747ec4568967f867c1d02acf5/artifacts', end_time=1732730282911, experiment_id='1', lifecycle_stage='active', run_id='9b8e79d747ec4568967f867c1d02acf5', run_name='abrasive-slug-522', run_uuid='9b8e79d747ec4568967f867c1d02acf5', start_time=1732730192208, status='FINISHED', user_id='codespace'>, inputs=<RunInputs: dataset_inputs=[]>>

In [11]:
for run in runs:
    print(f'run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}')

run id: 9b8e79d747ec4568967f867c1d02acf5, rmse: 5.2186
run id: ed80bfe153d44d5098e26483cf331c9a, rmse: 5.2212
run id: 4ae919aba7d04337a76c12fc214fdd31, rmse: 5.2319
run id: eb35fc11f0a840d4a9c0d042ee7ce67f, rmse: 5.3365
run id: cd1d9960f9a5482c918e1534903c9495, rmse: 5.4627


## Promote models to registry

In [16]:
import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

### Register new model

In [15]:
run_id = '9b8e79d747ec4568967f867c1d02acf5'
model_uri = f'runs:/{run_id}/model'

mlflow.register_model(model_uri=model_uri, name='nyc-taxi regressor')

Registered model 'nyc-taxi regressor' already exists. Creating a new version of this model...
Created version '5' of model 'nyc-taxi regressor'.


<ModelVersion: aliases=[], creation_timestamp=1736869500609, current_stage='None', description=None, last_updated_timestamp=1736869500609, name='nyc-taxi regressor', run_id='9b8e79d747ec4568967f867c1d02acf5', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/9b8e79d747ec4568967f867c1d02acf5/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=5>

### Transit a model

In [34]:
client.search_registered_models()

[<RegisteredModel: aliases={'challenger': 1, 'champion': 3}, creation_timestamp=1736525607132, description='NYC taxi predictor for trip duration', last_updated_timestamp=1736869500609, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1736869500609, current_stage='None', description=None, last_updated_timestamp=1736869500609, name='nyc-taxi regressor', run_id='9b8e79d747ec4568967f867c1d02acf5', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/9b8e79d747ec4568967f867c1d02acf5/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=5>], name='nyc-taxi regressor', tags={}>]

In [31]:
model_name = 'nyc-taxi regressor'
latest_versions = client.get_latest_versions(name=model_name)

/tmp/ipykernel_5457/2854498232.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [32]:
for version in latest_versions:
    print(f'version: {version.version}, alias: {version.aliases}')

version: 5, alias: []


In [37]:
model_version = 5
new_alias = 'test-alias'
client.set_registered_model_alias(
    name=model_name, 
    alias=new_alias, 
    version=model_version,
    )

In [39]:
from datetime import datetime

date = datetime.today().date()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f'The model version {model_version} was transitioned to {new_alias} on {date}'
)

<ModelVersion: aliases=['test-alias'], creation_timestamp=1736869500609, current_stage='None', description='The model version 5 was transitioned to test-alias on 2025-01-14', last_updated_timestamp=1736870773234, name='nyc-taxi regressor', run_id='9b8e79d747ec4568967f867c1d02acf5', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/9b8e79d747ec4568967f867c1d02acf5/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=5>